In [1]:
import gymnasium as gym
import numpy as np
from stable_baselines3.common.env_util import make_vec_env as make_vec_env_sb3
from copy import deepcopy
import torch
from torch.distributions import Normal
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.data import Dataset, DataLoader
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
import imageio
import pandas as pd
from tqdm import tqdm
from PIL import Image
from stable_baselines3.dqn.policies import MlpPolicy
import matplotlib.pyplot as plt


In [2]:
import gc
gc.collect()

7

In [2]:
!nvidia-smi
# clean up memory forcefully
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device



Tue Apr  2 17:55:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
|  0%   44C    P8              16W / 250W |      1MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

NameError: name 'torch' is not defined

In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
class CustomDataset(Dataset):
    def __init__(self, observations, actions):
        self.observations = torch.tensor(observations, dtype=torch.float32,device=device)
        self.actions = torch.tensor(actions, dtype=torch.float32,device=device)

    def __len__(self):
        return len(self.observations)

    def __getitem__(self, idx):
        observation = self.observations[idx]
        action = self.actions[idx]
        return observation, action

class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.norm1 = nn.LayerNorm(32)
        self.fc2 = nn.Linear(32, 32)
        self.norm2 = nn.LayerNorm(32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = F.relu(self.norm1(self.fc1(x)))
        x = F.relu(self.norm2(self.fc2(x)))
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    

Using device: cuda


In [5]:
env_id = "CartPole-v1"
# env = make_vec_env(env_id, n_envs=1)
# read /mnt/nfs/work/c98181/RL/dataset/CartPole-v1...npy
# np.save("/mnt/nfs/work/c98181/RL/dataset/"+env_id+f"_28000_obs.npy", obs_list)
# np.save("/mnt/nfs/work/c98181/RL/dataset/"+env_id+"_28000_actions.npy", actions_list)
# np.save("/mnt/nfs/work/c98181/RL/dataset/"+env_id+"_28000_rewards.npy", rewards_list)
# np.save("/mnt/nfs/work/c98181/RL/dataset/"+env_id+"_28000_dones.npy", dones_list)
# np.save("/mnt/nfs/work/c98181/RL/dataset/"+env_id+"_28000_info.npy", info_list)
# np.save("/mnt/nfs/work/c98181/RL/dataset/"+env_id+"_28000_next_obs.npy", next_obs_list)
observations= np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_28000_obs.npy", allow_pickle=True)
actions = np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_28000_actions.npy", allow_pickle=True)
rewards = np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_28000_rewards.npy", allow_pickle=True)
dones = np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_28000_dones.npy", allow_pickle=True)
info = np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_28000_info.npy", allow_pickle=True)
next_obs = np.load("/mnt/nfs/work/c98181/RL/dataset/CartPole-v1_28000_next_obs.npy", allow_pickle=True)
observations=observations.squeeze()
actions=actions.squeeze()
rewards=rewards.squeeze()
dones=dones.squeeze()
info=info.squeeze()
next_obs=next_obs.squeeze()

# observations = observations[:1000]
# actions = actions[:1000]
print(observations.shape, actions.shape, rewards.shape, dones.shape, info.shape, next_obs.shape)
print(observations[0], actions[0], rewards[0], dones[0], info[0], next_obs[0])

# print(actions)

(28000, 4) (28000,) (28000,) (28000,) (28000,) (28000, 4)
[-0.0455423  -0.00493775  0.01715994  0.02859923] 0 1.0 False {'TimeLimit.truncated': False} [-0.04564106 -0.20030153  0.01773192  0.32664654]


In [6]:
class mydataset(Dataset):
    def __init__(self, observations, actions):
        self.observations = torch.tensor(observations, dtype=torch.float32).to(device)
        self.actions = torch.tensor(actions, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.observations)

    def __getitem__(self, idx):
        observation = self.observations[idx]
        action = self.actions[idx]
        return observation, action

In [7]:
# import numpy as np
# import gymnasium as gym
# from imitation.policies.serialize import load_policy
# from imitation.util.util import make_vec_env
# from imitation.data.wrappers import RolloutInfoWrapper
# env = make_vec_env(
#     "seals:seals/CartPole-v0",
#     rng=np.random.default_rng(),
#     post_wrappers=[
#         lambda env, _: RolloutInfoWrapper(env)
#     ],  # needed for computing rollouts later
# )
# from stable_baselines3.common.evaluation import evaluate_policy

# # reward, _ = evaluate_policy(expert, env, 10)
# print(reward)

In [8]:
policy_network = PolicyNetwork(
    4, 2).to(device)

nn.init.kaiming_normal_(policy_network.fc1.weight)
nn.init.kaiming_normal_(policy_network.fc2.weight)
nn.init.kaiming_normal_(policy_network.fc3.weight)
prev = PolicyNetwork(
    4, 2).to(device)
prev.load_state_dict(policy_network.state_dict())
prev.eval()

optimizer = torch.optim.Adam(policy_network.parameters(), lr=0.0017601048183920826,weight_decay=2.350251568550711e-5)
env=make_vec_env_sb3(env_id, n_envs=1)

# Training loop
num_epochs = 1
batch_size = 16

dataset = mydataset(observations=observations, actions=actions)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

loss_graph=[]
eval_rewards=[]
margin_graph=[]
positive_reward_graph=[]
negative_reward_graph=[]
log_interval = 500
now_log=0
now_pos=0
for epoch in range(num_epochs):
    policy_network.train()

    # Compute the log probabilities of the actions
    pbar=tqdm(dataloader,position=0,leave=True)
    now_log=0

    for step, (obs_batch,act_batch) in enumerate(pbar):
        # obs_batch = torch.tensor(np.array(obs_batch), dtype=torch.float32).to(device)
        # act_batch = torch.tensor(np.array(act_batch), dtype=torch.float32).to(device)

        now_log+=obs_batch.shape[0]
        now_pos+=obs_batch.shape[0]
        
        logits = policy_network(obs_batch)
        model_dist = Categorical(logits=logits)
        model_act_sample = model_dist.sample()
        policy_chosen_logps = model_dist.log_prob(act_batch)
        policy_rejected_logps = model_dist.log_prob(model_act_sample)
        with torch.no_grad():
            prev_dist = Categorical(logits=prev(obs_batch))
            reference_chosen_logps = prev_dist.log_prob(act_batch)
            reference_rejected_logps = prev_dist.log_prob(model_act_sample)
        # print all logps
        # print("policy_chosen_logps",policy_chosen_logps.detach().mean().item())
        # print("policy_rejected_logps",policy_rejected_logps.detach().mean().item())
        # print("reference_chosen_logps",reference_chosen_logps.detach().mean().item())
        # print("reference_rejected_logps",reference_rejected_logps.detach().mean().item())

        pi_logratios = policy_chosen_logps - policy_rejected_logps
        ref_logratios = reference_chosen_logps - reference_rejected_logps
        # print("pi_logratios",pi_logratios.detach().mean().item())
        # print("ref_logratios",ref_logratios.detach().mean().item())
        # chosen_KL = (policy_chosen_logps - reference_chosen_logps).mean().clamp(min=0)
        # reject_KL = (policy_rejected_logps - reference_rejected_logps).mean().clamp(min=0)

        logits = pi_logratios - ref_logratios
        # print("logits",logits.detach().mean().item())

        chosen_logratios = policy_chosen_logps - reference_chosen_logps
        reject_logratios = policy_rejected_logps - reference_rejected_logps

        # logits = chosen_logratios - reject_logratios

        if epoch <= -1:
          loss = - (policy_chosen_logps).mean()
        else:
          beta = 1
          losses = (-F.logsigmoid(beta * logits))
          loss = losses.mean()
        #   losses = torch.cat((1 - F.logsigmoid(beta * (chosen_logratios - reject_KL)), 1 - F.logsigmoid(beta * (chosen_KL - reject_logratios))), 0)
        # Optimize the policy
        optimizer.zero_grad()

        prev.load_state_dict(policy_network.state_dict())
        prev.eval()

        loss.backward()

        optimizer.step()
        pbar.set_description((f"Epoch [{epoch+1}/{num_epochs}]"))
        positive_reward = chosen_logratios.detach().mean().item()
        negative_reward = reject_logratios.detach().mean().item()
        margin = positive_reward - negative_reward
        loss_graph.append(loss.detach().item())
        margin_graph.append(margin)
        positive_reward_graph.append(positive_reward)
        negative_reward_graph.append(negative_reward)
        pbar.set_postfix({"loss":loss.detach().item(), "positive_reward": positive_reward, "negative_reward": negative_reward, "margin": positive_reward - negative_reward})


        total_reward=0
        num_test=10
        if now_log>=log_interval:
            now_log-=log_interval
            eval_rewards.append(eval)
            policy_network.eval()  # 切换到评估模式

            

            total_reward=0
            # test the policy and save as gif
            frames = []
            for _ in range(num_test):
                state = env.reset()
                done = False
                while not done:
                    state_tensor = torch.tensor([state], dtype=torch.float32).to(device)
                    with torch.no_grad():
                        action = Categorical(logits=(policy_network(state_tensor))).sample().cpu().numpy()[0]
                        
                    state, reward, done, _ = env.step(action)  # 执行动作
                    total_reward += reward
                    frame = env.render(mode="rgb_array")  # 获取当前环境的图像
                    frames.append(frame)  # 添加到帧列表中
                # 保存为GIF
            image_path = f"cartpole_epoch_{now_pos}.gif"
            imageio.mimsave("/mnt/nfs/work/c98181/RL/result/"+env_id+"/"+image_path, frames)  # duration控制帧切换的速度
            print(total_reward/num_test)
            eval_rewards.append(total_reward)
            policy_network.train()  # 切换回训练模式


         

env.close()

Epoch [1/1]:   1%|▏         | 23/1750 [00:00<00:31, 55.13it/s, loss=0.692, positive_reward=0.00362, negative_reward=0.000447, margin=0.00318]/tmp/ipykernel_1291810/3251528753.py:117: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025847094/work/torch/csrc/utils/tensor_new.cpp:275.)
  state_tensor = torch.tensor([state], dtype=torch.float32).to(device)
Epoch [1/1]:   3%|▎         | 58/1750 [00:19<08:50,  3.19it/s, loss=0.692, positive_reward=0.000578, negative_reward=-.000952, margin=0.00153] 

[77.6]


Epoch [1/1]:   4%|▍         | 73/1750 [00:48<21:58,  1.27it/s, loss=0.697, positive_reward=-.00177, negative_reward=0.00553, margin=-.0073]    

[117.9]


Epoch [1/1]:   6%|▋         | 110/1750 [01:18<19:45,  1.38it/s, loss=0.686, positive_reward=0.0103, negative_reward=-.00513, margin=0.0154]     

...

[306.]


Epoch [1/1]:  31%|███▏      | 549/1750 [12:24<30:30,  1.52s/it, loss=0.692, positive_reward=0.000127, negative_reward=-.00169, margin=0.00182]  

[259.3]


Epoch [1/1]:  33%|███▎      | 581/1750 [13:26<30:06,  1.55s/it, loss=0.697, positive_reward=-.00409, negative_reward=0.00365, margin=-.00774]  

[291.6]


In [ ]:
# draw the loss graph
import matplotlib.pyplot as plt
plt.plot(loss_graph)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Loss vs Epoch")
plt.show()

# draw the reward graph
plt.plot(eval_rewards)
plt.xlabel("Iteration")
plt.ylabel("Reward")
plt.title("Reward vs Iteration")
plt.show()

# drqw the margin graph
plt.plot(margin_graph)
plt.xlabel("Iteration")
plt.ylabel("Margin")
plt.title("Margin vs Iteration")
plt.show()

# draw the positive reward graph
plt.plot(positive_reward_graph)
plt.xlabel("Iteration")
plt.ylabel("Positive Reward")
plt.title("Positive Reward vs Iteration")
plt.show()

# draw the negative reward graph
plt.plot(negative_reward_graph)
plt.xlabel("Iteration")
plt.ylabel("Negative Reward")
plt.title("Negative Reward vs Iteration")
plt.show()